In [1]:
# работа с данными
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

# нормализация
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures

from scipy.stats import skew, kurtosis
from scipy.stats import trim_mean, trimboth, tmean, tstd, ttest_1samp, gmean, hmean, mode
from scipy.stats import percentileofscore, scoreatpercentile
import statistics

from collections import Counter

In [69]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.linear_model import Lasso 

In [68]:
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats.distributions import randint

# ФУНКЦИИ

In [59]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)    
    return X_meta_train, X_meta_test


def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])
    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])    
    return stacked_features_train, stacked_features_test


def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_test_pred)
    f1 =  f1_score(y_test, y_test_pred, average='macro')
    return accuracy, f1

# Загрузка ds

In [103]:
f_1 = ['id', 'distance', 'Кол-во пиков', 'Сред.зн.пика', 'Медиана', 'Смещение', 
       'Ниж.квартиль', 'Верх.квартиль', 'Дисперсия', 'Куртозис', 'X0', 'X1', 'X2']
f_2 = ['Max', 'Mean', 'Std откл.', 'Асимметрия', 'Куртозис', 'id']

feature_3 = ['Max', 'Куртозис', 'Гарм.сред.', 'Std откл.', 'Асимметрия', 'Ниж.квартиль', 'Верх.квартиль', 'id'] 
feature_4 = ['Max', 'Куртозис', 'Гарм.сред.', 'Std откл.', 'Асимметрия', 'X0', 'X1', 'X2', 'id'] 
feature_5 = ['Max', 'Куртозис', 'Гарм.сред.', 'Std откл.', 'Асимметрия', 'distance', 'id'] 
feature_6 = ['Max', 'Куртозис', 'Гарм.сред.', 'Std откл.', 'Асимметрия', 'Кол-во пиков', 'Сред.зн.пика', 'id'] 
feature_7 = ['Max', 'Куртозис', 'Гарм.сред.', 'X0', 'X1', 'X2', 'distance', 'Кол-во пиков', 'Сред.зн.пика', 'id'] 
feature_8 = ['Max', 'Куртозис', 'Гарм.сред.', 'Std откл.', 'Асимметрия', 'X0', 'X1', 'X2', 'distance', 'id'] 

In [88]:
file_name_dum_balance = './datasets/all_dataset_dum_balance.csv'
data_balance = pd.read_csv(file_name_dum_balance, sep=";", encoding='utf8') #, low_memory=False)
data_balance = data_balance.drop(['direction'], axis=1)
data_balance.head(2)

,id,distance,Кол-во пиков,Сред.зн.пика,Min,Max,Медиана,Смещение низ,Смещение верх,Смещение,...,Верх.квартиль,Дисперсия,Асимметрия,Куртозис,Std откл.,Коэф.вариации,Std ошибка,X0,X1,X2
0,1,6656,216,530.898,3.836,7.216,5.428,1.592,1.788,47.102,...,6.157,0.660,0.033,-1.112,0.812,0.149,0.025,1,0,0
1,1,6625,216,532.741,3.226,6.143,4.719,1.493,1.424,51.180,...,5.323,0.555,0.024,-1.114,0.745,0.159,0.023,1,0,0


In [23]:
file_name_dum = './datasets/all_dataset_dum.csv'
data = pd.read_csv(file_name_dum, sep=";", encoding='utf8') #, low_memory=False)
data = data.drop(['direction'], axis=1)
data.head(2)

,id,distance,len,Кол-во пиков,Сред.зн.пика,Min,Max,Медиана,Смещение низ,Смещение верх,...,Верх.квартиль,Дисперсия,Асимметрия,Куртозис,Std откл.,Коэф.вариации,Std ошибка,X0,X1,X2
0,1,6656,1077,216,530.898,3.836,7.216,5.428,1.592,1.788,...,6.157,0.660,0.033,-1.112,0.812,0.149,0.025,1,0,0
1,1,6625,1077,216,532.741,3.226,6.143,4.719,1.493,1.424,...,5.323,0.555,0.024,-1.114,0.745,0.159,0.023,1,0,0


# Стекинг

### data[f_2]

In [45]:
df = data[f_2]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

(3972, 5)


In [63]:
classifiers = [
    # LogisticRegression(C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000, random_state=42),
    LogisticRegression(max_iter=2000, random_state=42),
    RandomForestClassifier(n_estimators=300, random_state=42),
    GradientBoostingClassifier(n_estimators=200, random_state=42)
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

# Создание мета-алгоритма и оценка его качества
meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:07<00:00, 22.34s/it]

acc = 70.39% f1 = 0.69%


In [64]:
classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.70s/it]

acc = 71.00% f1 = 0.70%


In [66]:
classifiers = [
    LogisticRegression(max_iter=5000),
    KNeighborsClassifier(),
    RandomForestClassifier(n_estimators=300, random_state=42),
    AdaBoostClassifier()
]

cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv)

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:24<00:00,  6.11s/it]

acc = 70.19% f1 = 0.69%


In [76]:
classifiers = [
    RandomForestClassifier(n_estimators=300, random_state=42),
    ExtraTreesClassifier(n_estimators=300, random_state=42)
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
features = []

for clf in tqdm(classifiers):
    X_meta_train = np.zeros((len(X_train), len(np.unique(y_train))), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), len(np.unique(y_train))), dtype=np.float32)
    
    for train_fold_index, predict_fold_index in cv.split(X_train, y_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test += folded_clf.predict_proba(X_test)
    
    X_meta_test /= 5  # Усреднение по фолдам
    features.append((X_meta_train, X_meta_test))

# Создание мета-алгоритма и оценка его качества
stacked_features_train = np.hstack([f[0] for f in features])
stacked_features_test = np.hstack([f[1] for f in features])

meta_clf = RandomForestClassifier(random_state=42)
meta_clf.fit(stacked_features_train, y_train)

y_pred_meta = meta_clf.predict(stacked_features_test)
acc = accuracy_score(y_test, y_pred_meta)
f1 =  f1_score(y_test, y_pred_meta, average='macro')
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.43s/it]


acc = 66.57% f1 = 0.66%


In [106]:
df = data[feature_5]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3972, 6)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.56s/it]

acc = 74.52% f1 = 0.74%


In [112]:
df = data[f_1]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3972, 12)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.24s/it]

acc = 75.03% f1 = 0.75%


### data_balance

In [91]:
df = data_balance[f_2]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)



  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.86s/it]

acc = 68.48% f1 = 0.68%


In [97]:
df = data_balance[f_1]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3385, 12)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.17s/it]

acc = 78.39% f1 = 0.78%


In [110]:
df = data_balance[f_1]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    LogisticRegression(max_iter=2000, random_state=42),
    RandomForestClassifier(n_estimators=300, random_state=42),
    # GradientBoostingClassifier(n_estimators=200, random_state=42),
    # KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    # ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3385, 12)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

 50%|████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                | 1/2 [00:02<00:02,  2.26s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:46<00:00, 23.10s/it]

acc = 76.27% f1 = 0.76%


In [111]:
df = data_balance[feature_5]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3385, 6)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.63s/it]

acc = 75.32% f1 = 0.75%


In [104]:
df = data_balance[feature_8]
y =  np.array(df['id'])  # Target variable
X = np.array(df.drop('id', axis=1))  # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)

classifiers = [
    KNeighborsClassifier(),  # Метод ближайших соседей со стандартными параметрами
    ExtraTreesClassifier(n_estimators=300, random_state=42)  # Случайный лес из 300 экстремальных деревьев
]

cv = KFold(n_splits=10, shuffle=True, random_state=42) # Кросс-валидация
X_meta_train, X_meta_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv) # Генерация мета-признаков

meta_clf = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
meta_clf.fit(X_meta_train, y_train)
acc, f1 = compute_metric(meta_clf, X_meta_train, y_train, X_meta_test)
print("acc = %.2f%%" % (acc*100), 'f1 = %.2f%%' % f1)

(3385, 9)




  0%|                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.76s/it]

acc = 76.51% f1 = 0.76%
